# Crawl weather on China meteorological administration Website

### data source: http://www.weather.com.cn/textFC/hb.shtml
### Goal: crawl temperature information of every area,including hb,db,hd, in china and develop a website with a bar chart of top 10 lowest tempereature cities

In [7]:
from bs4 import BeautifulSoup
import requests
from pyecharts import Bar

ALL_DATA = []

HEADER = {
    'User-Agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.92 Safari/537.36"
}

def main():

    urls = [
    'http://www.weather.com.cn/textFC/hb.shtml',
    'http://www.weather.com.cn/textFC/db.shtml',
    'http://www.weather.com.cn/textFC/hd.shtml',
    'http://www.weather.com.cn/textFC/hz.shtml',
    'http://www.weather.com.cn/textFC/hn.shtml',
    'http://www.weather.com.cn/textFC/xb.shtml',
    'http://www.weather.com.cn/textFC/xn.shtml',
    'http://www.weather.com.cn/textFC/gat.shtml'

    ]
    for url in urls:
        parse_page(url)

    ALL_DATA.sort(key=lambda x:x['min_temp'] )
    #print(ALL_DATA)

    data = ALL_DATA[0:10]

    cities = list( map(lambda x:x['city'],data))
    temps = list(map(lambda x:x['min_temp'],data))

    chart = Bar('Chinese lowest temperature ranking')
    chart.use_theme('dark')
    chart.add('',cities,temps)
    chart.render('temperature.html')

def parse_page(url):
    response = requests.get(url,headers = HEADER)
    html = response.content.decode('utf-8')
    soup = BeautifulSoup(html,'html5lib')
    conMidtab = soup.find('div',class_ = 'conMidtab')
    tables = conMidtab.find_all('table')
    for table in tables:
        trs = table.find_all('tr')[2:]
        for index,tr in enumerate(trs):
            tds = tr.find_all('td')
            city_td = tds[0]
            if index == 0:
                city_td = tds[1]
            city = list(city_td.stripped_strings)[0]
            temp_td = tds[-2]
            min_temp = list(temp_td.stripped_strings)[0]
            ALL_DATA.append({'city': city, 'min_temp': int(min_temp)})
           # print({'city': city, 'min_temp': int(min_temp)})
            #print("="*30)
    



if __name__ == '__main__':
    # sorted by temperature descendingly
    main()
